In [175]:
%env CUDA_VISIBLE_DEVICES='-1'
import tensorflow as tf
import numpy as np
import pandas as pd
import glob
import os
from IPython.display import display, Audio

env: CUDA_VISIBLE_DEVICES='-1'


In [176]:
vocab = pd.read_csv('train_gamemaster/vocab.csv', header=None, index_col=0, squeeze=True).astype(np.int32)
print(vocab)

0
movie            0
alarm            1
chair            2
pencil           3
cow              4
seagul           5
chimes           6
unsheathe        7
coin             8
hospital         9
gorilla         10
elec            11
lever           12
cartoon         13
river           14
typing          15
background      16
grasshopper     17
water           18
printer         19
cork            20
crow            21
dirt            22
radio           23
engine          24
phone           25
healing         26
fear            27
seal            28
bullet          29
              ... 
voice          150
time           151
footstep       152
zebra          153
nature         154
bear           155
elevator       156
boiling        157
taser          158
duck           159
metallic       160
wolf           161
tas            162
sword          163
crickets       164
zombie         165
chicken        166
monkey         167
cash           168
announcer      169
metal          170
points    

In [301]:
tf.reset_default_graph()

saver = tf.train.import_meta_graph('train_gamemaster/infer/infer.meta')
graph = tf.get_default_graph()
sess = tf.InteractiveSession()
latest_ckpt_fp = tf.train.latest_checkpoint('train_gamemaster')
saver.restore(sess, latest_ckpt_fp)
num_samples = 10

gan_embed_table = graph.get_tensor_by_name('G/embed_table:0')
index = np.random.randint(len(vocab))
yembed = tf.nn.embedding_lookup(gan_embed_table, [index])
# yembed = 0.5 * tf.nn.embedding_lookup(gan_embed_table, [vocab.loc['hippo']]) + 0.5 * yembed
yembed = tf.tile(yembed, [num_samples, 1])
_yembed = sess.run(yembed)

_z = tf.truncated_normal([num_samples, 100], stddev=1)
_z = sess.run(_z)

z = graph.get_tensor_by_name('z:0')
yembed_place_holder = graph.get_tensor_by_name('yembed:0')
_G_z = sess.run(graph.get_tensor_by_name('G_z:0'), {z: _z, yembed_place_holder: _yembed})

INFO:tensorflow:Restoring parameters from train_gamemaster\model.ckpt-58571


In [302]:
print(vocab.index[index])
for audio in _G_z:
    display(Audio(audio.T, rate=16000))

foley
